# \[7\] Classification

In [ ]:
import os, matplotlib, csv, shutil, json, random
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from IPython.display import Image
import skimage
from skimage import io as skio
from skimage import transform as skit

# scikit-learn
import sklearn.metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

# https://pytorch.org/vision/stable/datasets.html
# 파이토치에서는 torchvision.datasets에 MNIST 등의 다양한 데이터를 사용하기 용이하게 정리해놨습니다.
# 이를 사용하면 데이터를 따로 학습에 맞게 정리하거나 하지 않아도 바로 사용이 가능합니다.
import torchvision.datasets as dset

# https://pytorch.org/vision/stable/transforms.html
# torchvision.transforms에는 이미지 데이터를 자르거나 확대 및 다양하게 변형시키는 함수들이 구현되어 있습니다.
import torchvision.transforms as transforms

# https://pytorch.org/docs/stable/data.html
# DataLoader는 전처리가 끝난 데이터들을 지정한 배치 크기에 맞게 모아서 전달해주는 역할을 합니다.
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

print(torch.__version__)

## 1. VGG

### 1.1. Data Loading
* Kaggle Garbage Classification Dataset
  - Reference link: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification

In [ ]:
# Dataset Download
!git clone https://github.com/Pulsar-kkaturi/DL-Education.git

In [ ]:
# 데이터 압축 풀기
!tar -zxf ./DL-Education/dataset/garbage_cls_2d.tar.gz

In [ ]:
data_path = './garbage_cls_2d'
label_list = list(sorted(os.listdir(data_path)))
print(label_list)

In [ ]:
# data number check
for c in label_list:
  cn = 0
  cls_path = os.path.join(data_path, c)
  for f in os.listdir(cls_path):
    cn += 1
  print(f'{c} number = {cn}')

In [ ]:
# Data visulization
plt.figure(figsize=(12,6))
for i, c in enumerate(label_list):
  cls_path = os.path.join(data_path, c)
  sam_path = os.path.join(cls_path, os.listdir(cls_path)[0])
  sam_arr = skio.imread(sam_path)
  plt.subplot(2,3,i+1)
  plt.title(c)
  plt.imshow(sam_arr)

### 1.2. Data Preprocessing

#### 1.2.1. Data Processing 

In [ ]:
# one-hot encoding
for c in label_list:
  onehot = [0]*len(label_list)
  onehot[label_list.index(c)] = 1
  print(c, label_list.index(c), onehot)

In [ ]:
# Numpy array로 데이터 변환
data_dic, label_dic = {}, {}
for key in label_list:
  data_dic[key] = []
  label_dic[key] = []

for i, c in enumerate(label_list):
  cls_path = os.path.join(data_path, c)
  for f in sorted(os.listdir(cls_path)):
    file_path = os.path.join(cls_path, f)
    data_arr = skio.imread(file_path)
    onehot = [0]*len(label_list)
    onehot[label_list.index(c)] = 1
    data_dic[c].append(data_arr)
    label_dic[c].append(onehot)

In [ ]:
plt.figure(figsize=(12,6))
for i, k in enumerate(data_dic.keys()):
  print(f'{k}: data_number = {len(data_dic[k])}, label_number = {len(label_dic[k])}')
  plt.subplot(2,3,i+1)
  plt.title(f'{k}: {label_dic[k][0]}')
  plt.imshow(data_dic[k][0])

#### 2.2. Tensor Setting